<a href="https://colab.research.google.com/github/bethel2230/netflixdataanalysis/blob/main/netflixdataanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #linearalgebra
import pandas as pd #data preparation
import plotly.express as px #data visualization
from textblob import TextBlob #sentiment analysis
from scipy import stats

df = pd.read_csv('netflix_titles.csv') #read the data into the document

In [ ]:
df.shape #rows and columns in data set

(8807, 12)

In [ ]:
df.head() #first five rows of data set




,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
x = df.groupby(["rating"]).size().reset_index(name = 'Appearances') #command to group content by rating and appearances
print(x)


      rating  Appearances
0     66 min            1
1     74 min            1
2     84 min            1
3          G           41
4      NC-17            3
5         NR           80
6         PG          287
7      PG-13          490
8          R          799
9      TV-14         2160
10      TV-G          220
11     TV-MA         3207
12     TV-PG          863
13      TV-Y          307
14     TV-Y7          334
15  TV-Y7-FV            6
16        UR            3


In [ ]:
pieChart = px.pie(x, values = "Appearances", names = "rating", title = "Distribution of Title ratings on Netflix")
pieChart.show()

In [ ]:

df = pd.read_csv("netflix_titles.csv")

# Define default values for each column
defaults = {
    "director": "Director not specified",
    "cast": "Cast not specified",
    "country": "Country not specified",
}

# Fill missing values using the defaults dictionary
df.fillna(defaults, inplace=True)

# Show the modified DataFrame
df.head()



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Cast not specified,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Director not specified,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Country not specified,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,Director not specified,Cast not specified,Country not specified,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,Director not specified,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
directors_list = pd.DataFrame()
directors_list = df['director'].str.split(',', expand=True).stack()
directors_list = directors_list.to_frame()
directors_list.columns = ['Director']
directors = directors_list.groupby(['Director']).size().reset_index(name='Total Count')
directors = directors[directors.Director != 'Director not specified']
directors = directors.sort_values(by=['Total Count'], ascending = False)
top5Directors = directors.head()
top5Directors = top5Directors.sort_values(by=['Total Count'])
barChart = px.bar(top5Directors, x='Total Count', y = 'Director', title = 'Top 5 Directors on Netflix')
barChart.show()


In [ ]:

df3 = df[['release_year', 'description']]
df3 = df3.rename(columns = {'release_year':'Release Year', 'description':'Description'})
for index, row in df3.iterrows():
  d=row['Description']
  testimonial = TextBlob(d)
  p = testimonial.sentiment.polarity
  if p==0:
    sent = 'Neutral'
  elif p>0:
    sent = 'Positive'
  else:
    sent = 'Negative'
  df3.loc[[index, 2], 'Sentiment']=sent

df3 = df3.groupby(['Release Year', 'Sentiment']).size().reset_index(name = 'Total Count')

df3 = df3[df3['Release Year']>2005]
barGraph = px.bar(df3, x="Release Year", y="Total Count", color = "Sentiment", title = "Sentiment Analysis of Content on Netflix")
barGraph.show()

In [ ]:

df['country'] = df['country'].str.split(', ')
df = df.explode('country')

top_countries = df['country'].value_counts().nlargest(5) #n largest for specific count
print(top_countries)



United States     3689
India             1046
United Kingdom     804
Canada             445
France             393
Name: country, dtype: int64


In [ ]:
fig = px.pie(
    names=top_countries.index,
    values=top_countries,
    title='Top 5 Countries with the Most Titles on Netflix'
)

fig.show()